In [1]:
from Game.BlackJack import BlackJack
from Agent.random.randomagent import RandomAgent
from Agent.nn.nnagent import nnAgent
from Agent.LLM.LLM_Agent import LLM
from Agent.qlearning.qLearningAgent import qLearningAgent

Training progress: 0.00%
Training progress: 1.00%
Training progress: 2.00%
Training progress: 3.00%
Training progress: 4.00%
Training progress: 5.00%
Training progress: 6.00%
Training progress: 7.00%
Training progress: 8.00%
Training progress: 9.00%
Training progress: 10.00%
Training progress: 11.00%
Training progress: 12.00%
Training progress: 13.00%
Training progress: 14.00%
Training progress: 15.00%
Training progress: 16.00%
Training progress: 17.00%
Training progress: 18.00%
Training progress: 19.00%
Training progress: 20.00%
Training progress: 21.00%
Training progress: 22.00%
Training progress: 23.00%
Training progress: 24.00%
Training progress: 25.00%
Training progress: 26.00%
Training progress: 27.00%
Training progress: 28.00%
Training progress: 29.00%
Training progress: 30.00%
Training progress: 31.00%
Training progress: 32.00%
Training progress: 33.00%
Training progress: 34.00%
Training progress: 35.00%
Training progress: 36.00%
Training progress: 37.00%
Training progress: 38.

In [2]:
# agent = nnAgent()
game = BlackJack("novel")

In [3]:
for i in range(5):
    agent = qLearningAgent()
    game.start()
    print("Dealer shows:", game.format_cards(game.dealer_hand[:1]))
    while game.status == "continue":
        dealer_card = game.total_value(game.dealer_hand[:1])
        action = agent.choose_action(game.get_playervalue(),dealer_card,qLearningAgent.has_usable_ace(game.player_hand))
        game.play(action, "basic")
    print(f"dealer's hand: {game.format_cards(game.dealer_hand)} {game.get_dealervalue()}  player's hand: {game.format_cards(game.player_hand)} {game.get_playervalue()}")
    print(game.game_result())
    
    if len(game.deck) < 20:
        game.reset()

Dealer shows: 7♥
dealer's hand: 7♥ 6♠ 5♠ 18  player's hand: 8♥ Q♣ 18
draw
Dealer shows: 7♣
dealer's hand: 7♣ 10♦ 17  player's hand: 9♥ 6♣ 15
loss
Dealer shows: 8♠
dealer's hand: 8♠ K♣ 18  player's hand: 9♠ A♥ 7♠ 17
loss
Dealer shows: 3♦
dealer's hand: 3♦ 4♣ 5♣ 4♥ Q♦ 26  player's hand: 6♥ 5♦ 11
win
Dealer shows: 2♥
dealer's hand: 2♥ 3♥ 10♣ Q♥ 25  player's hand: 9♦ 2♠ 11
win
